In [1]:
import collections
from collections import defaultdict
from sklearn import svm
from sklearn import cross_validation
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
import pickle
import json

In [2]:
def get_words(docs):
    words = []
    for doc in docs:
        for word in doc.split():
            words.append(word)
    print ("The number of words: "+ str(len(set((words)))))
    return words

# define a funciton to find words occurring less than n times and delete that word
def findWordLessThan(counter, n):
    uncommon = []
    for k, v, in counter.items():
        if v < n:
            uncommon.append(k)            
    print (len(uncommon))
    return uncommon

def findWordMoreThan(counter, n):
    common = []
    for k, v, in counter.items():
        if v > n:
            common.append(k)
    print (len(common))
    return common

# Clean Reddit data

In [3]:
# specify dir
Dir = 'D:/Depression_project/data/reddit'

In [4]:
text_control = []
with open(Dir+'/'+'randomSubreddits2017_selected.txt',"r") as f:
    data = f.readlines()
text_control = [x.strip() for x in data] 
print (len(text_control))
text_control[:2]

98339


['want see how would address what predat eat food none charact eat meat entir movi be slightli bother bojack horseman do fantast dark episod show differ chicken eat chicken that be function member societi',
 'how tall be']

In [5]:
# read data
text_depr = []
with open(Dir+'/'+'mentalSubreddits2017_selected.txt',"r") as f:
    data = f.readlines()
text_depr = [x.strip() for x in data] 
print (len(text_depr))
text_depr[:3]

98745


['anybodi els look find reason live can not find',
 'bad part be have live crazi fuck world',
 'what do everyon do their free time summer break colleg blue seemingli my friend can not stand long summer break colleg bit month leav lot free time even work long shift week work anybodi els find case be say anybodi have idea how can fill my free time there be onli so much exercis netflix cook person can do haha']

# Clean control posts

In [6]:
def examineCorpus(raw_corpus, uncommon_cutoff, common_cutoff, messy_contain, messy_exact):
    count = collections.Counter(raw_corpus)
    uncommon = findWordLessThan(count, uncommon_cutoff)
    common = findWordMoreThan(count, common_cutoff)
    corpus = list(set(count.keys()) - set(uncommon))
    
    messy_common = []
    for w in corpus:
        for i in messy_contain:
            if w.find(i) != -1 or w in messy_exact or w.isdigit() == 1:
                messy_common.append(w)
    #print (messy_common)
    corpus = list(set(count.keys()) - set(uncommon) - set(messy_common))
    print ("The number of unique words is "+str(len(set(corpus))))
    return corpus

In [7]:
messy = ['|', '@', '_', '^', 'amp;', 'gt;', 'faq_autotldr_bot/', "](/r", '’', '-', '█', '%', '/r', '*', '°', 'lt;3', \
        '#', '?', '/','‘']
messy2 = ['a','b','c','d','e','f','g','h','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'] +\
['a.','b.','c.','d.','e.','f.','g.','h.','i.','j.','k.','l.','m.','n.','o.','p.','q.','r.','s.','t.','u.','v.','w.','x.','y.','z.']+\
["'",'.','+','(', ')','*']

In [8]:
words_control = get_words(text_control)

The number of words: 68146


In [9]:
corpus0 = examineCorpus(words_control, 11, 700, messy, messy2)

58919
435
The number of unique words is 8992


In [174]:
"""""
# check
for k in corpus0[4000:]:
    print (k)

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-174-107ef9a4ee06>, line 4)

In [ ]:
#########################    Ignore   #######################################

In [ ]:
words_control = get_words(text_control)
count = collections.Counter(words_control)

In [ ]:
uncommon = findWordLessThan(count, 11)
common = findWordMoreThan(count, 700)

In [ ]:
"""""
# check
for k, v in count.items():
    if k in common:
        print (k, v)

In [ ]:
corpus = list(set(count.keys()) - set(uncommon))

In [ ]:
messy = ['|', '@', '_', '^', 'amp;', 'gt;', 'faq_autotldr_bot/', "](/r", '’', '-', '█', '%', '/r', '(', ')', '*', '°', 'lt;3' \
        '#']
messy2 = ['a','b','c','d','e','f','g','h','i.','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
messy_common = []
for w in corpus:
    for i in messy:
        if w.find(i) != -1 and w not in messy2 and w.isdigit() != 1:
            messy_common.append(w)
print (len(messy_common))

In [ ]:
corpus0 = list(set(count.keys()) - set(uncommon) - messy_common)
print (len(corpus0))

In [ ]:
for k, v in collections.Counter(corpus0).items():
    if k in corpus[1000:2000]:
        print (k)

In [ ]:
########################        Ignore     ##############################################

# Clean depressed posts

In [10]:
words_depr = get_words(text_depr)

The number of words: 46498


In [ ]:
"""""
# check
for k, v in count.items():
    if k in uncommon:
        print (k, v)

In [11]:
messy = ['|', '_', '^', 'amp;', 'gt;', '’', '-', '█', '%', '/r', '?', '~', '.', '/', '‘', '#']
messy2 = ['a','b','c','d','e','f','g','h','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'] + \
['a.','b.','c.','d.','e.','f.','g.','h.','i.','j.','k.','l.','m.','n.','o.','p.','q.','r.','s.','t.','u.','v.','w.','x.','y.','z.']+\
["'",'.','+','=', '―','☆', '(', ')']

In [12]:
corpus1 = examineCorpus(words_depr, 5, 100, messy, messy2)

30933
3876
The number of unique words is 15143


In [13]:
# check
for k in corpus1[:4000]:
    print (k)

blu
argentina
improb
micro
os
nigger
ahhhh
deriv
pfft
rogan
brainless
airway
podcast
latter
sidewalk
swirl
neurofeedback
do't
oooh
transferr
transcrani
aircraft
viru
greec
kk
agonist
probali
curat
dublin
posibl
task
but
amantadin
blackout
issabel
scurri
vu
mvp
uninstal
pari
here
counselor
splendor
adequ
explor
cmht
friendless
noteworthi
beemind
pars
mystic
augment
carpent
sat
tradit
wan't
confort
thailand
museum
selfworth
satir
modul
17th
coop
irma
set
princ
eerili
indel
mr
bra
bing
barbarian
tens
ritz
moth
dealership
shakey
hibern
conclud
femin
competit
bpiier
pitch
xxx
shaki
aswel
crop
mandat
preoccupi
wine
nbd
sexism
misdiagnosi
ting
foggi
flamboy
structur
crutch
paul
yell
noob
choic
nitrogen
plung
hundr
rubric
abandon
wateri
monotoni
extremli
macro
deceit
gratifi
deeper
uncertainti
prolli
chagrin
rose
transfus
accuraci
niggl
guis
forego
aluminum
b12
justif
surpress
rethink
viral
experiment
gloat
haul
fyi
process
pco
smolder
spotless
pedestrian
anythign
overhaul
scrub
conflict
smoot

# Clean tweets

In [23]:
# specify dir
Dir = 'D:/Depression_project/data/twitter'

In [144]:
texts = []
ids = []
#with open(Dir+'/'+'tweets_cleaned.txt',"r", errors = 'ignore') as f:
with open(Dir+'/'+'tweetsEng_cleaned_2.txt',"r", errors = 'ignore') as f:
    data = f.readlines()
texts = [x.strip().split('#^~^#')[1] for x in data] 
ids = [x.strip().split('#^~^#')[0] for x in data] 
print (len(texts))
texts[:2]

171514


['partli cloudi night temp:3.4c hum:87 wind ene mph maxgust:10 mph rain mm baro:1010.6 hpa amp rise slowli weather',
 'rt date would enjoy']

In [25]:
words_twitter = get_words(texts)

The number of unique words: 131316


In [194]:
messy = ['#', '’', '@', 'rt', '_',  'http', '%', 'x', '…', '|', '/', '▔', 'amp', '┏', '★', '═', 'gt;',';&','lt;','】','↙'\
        '！', '+','╝','╔', '╚', '━', '║', '『','╮','╭', '°', '―', '█','◆', '┃','「','-','╱', '┓','▏', '╗', '─',\
         '┛','╲','」','“','┻','┗',',']
messy2 = ['a','b','c','d','e','f','g','h','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'] + \
['a.','b.','c.','d.','e.','f.','g.','h.','i.','j.','k.','l.','m.','n.','o.','p.','q.','r.','s.','t.','u.','v.','w.','x.','y.','z.']+\
["'",'.','+','×','=', ':','マエケン', '?', '偶像练习生','rt']

In [195]:
corpus_tw = examineCorpus(words_twitter, 10, 100, messy, messy2)

120912
1955
The number of unique words is 8272


In [202]:
for w in corpus_tw[:2000]:
    print (w)

revolut
thief
warehous
bridg
stunt
mandela
abigail
have
abus
enforc
madison
herb
postcard
heck
predat
instantli
hopeless
austin
manag
talaga
assur
eyebrow
time
p.m.
advocaci
ama
overr
thi
flyer
stadium
houston
drank
uniti
moonbeam
ahh
stitch
bent
honest
spray
barbara
salad
transit
rider
madam
carey
job
sou
home
sign
hr
troll
quiz
nose
dibasa
sun
grate
danisnot
daydream
mock
hehe
bath
pub
rosé
unicorn
mutual
kansa
dumbass
said
follo
dictionari
offici
fisa
town
incr
gam
guilt
salmon
casino
hugh
fave
audio
fansign
useless
slack
festiv
reset
kinda
vinyl
fbpe
doug
stack
altonsterl
atm
warmin
unseen
laser
hunger
saniti
reckon
cbd
magnific
affect
percept
fell
ceremoni
malaysian
gho
solat
amin
manli
jfc
nervou
pinch
kilikiligo
the
newspap
dog
wwe
provinc
rock
mankind
factori
kihyun
minhyuk
price
gc
critic
superfici
mosquito
wallpap
sergeant
mullet
toll
liar
knock
philadelphia
crybabi
renjun
caution
laugehd
layer
superjunior
tokensal
rot
cours
inc.
yukhei
backward
davi
leg
notion
everybodi
fbr


# Compare three collections of corpus

In [14]:
# check
d = collections.Counter(words_control)
s0 = [(k, d[k]) for k in sorted(d, key=d.get, reverse=True)]
for k, v in s0[:1000]:
    if k in corpus0:
        print (k, v)

be 165517
not 52392
have 43477
do 38555
get 17514
my 16091
can 15424
would 15010
just 14795
your 14597
will 13811
go 11677
what 11656
good 11562
so 10826
that 10753
there 10402
think 10176
make 10130
more 9517
peopl 9088
say 8273
when 8201
time 7914
see 7737
know 7537
use 7526
how 7367
want 6588
realli 6347
their 6335
other 6243
game 6199
onli 6127
hi 6113
then 5963
look 5937
who 5828
thing 5823
even 5822
also 5627
take 5552
work 5386
need 5289
much 5239
could 5227
now 5031
way 4766
tri 4740
play 4710
year 4580
still 4579
should 4561
which 4513
too 4453
post 4442
all 4320
feel 4220
whi 4158
right 4138
may 4116
as 4101
thank 3982
give 3979
like 3929
come 3879
veri 3857
mean 3774
someth 3742
same 3660
most 3634
here 3602
actual 3592
first 3582
find 3541
day 3516
lot 3454
back 3342
bad 3302
point 3285
where 3270
never 3202
sure 3165
one 3129
fuck 3126
question 3101
new 3068
pretti 3066
well 3064
love 3028
start 3022
it 2998
help 2867
guy 2825
great 2764
gt 2756
person 2727
our 2721
differ

In [15]:
# check
d = collections.Counter(words_depr)
s1 = [(k, d[k]) for k in sorted(d, key=d.get, reverse=True)]
for k, v in s1[:1000]:
    if k in corpus1:
        print (k, v)

be 975734
have 372748
not 365465
my 364999
do 310856
feel 155889
just 143492
get 129053
can 108436
go 105645
so 104948
know 87825
want 85068
what 78630
time 73683
depress 68670
would 67310
will 66550
think 66150
make 65607
when 65478
life 64196
how 62523
that 58738
good 58322
thing 56986
realli 56777
even 55765
day 55525
now 55479
peopl 55160
year 55132
tri 49257
friend 48972
work 48948
take 48269
help 47843
there 45156
say 44133
more 40851
start 38715
tell 38360
see 37463
bad 36542
much 36121
never 35909
could 35328
talk 35191
who 34606
all 34356
need 33955
other 33829
onli 33487
then 31496
someth 30746
back 29716
anyth 28922
anyon 28630
way 28262
still 28142
find 27857
fuck 27200
alway 26616
school 26047
come 25947
too 25418
love 24973
live 24914
your 24364
week 24298
veri 24286
someon 23991
month 23987
whi 23930
person 23653
keep 23322
job 23214
everyth 23011
also 22974
which 22641
here 22270
happi 22110
give 21961
last 21859
look 21788
where 21777
end 21158
again 20871
lot 20369
ri

In [187]:
# check
d = collections.Counter(words_twitter)
s = [(k, d[k]) for k in sorted(d, key=d.get, reverse=True)]
for k, v in s[1000:2000]:
    if k in corpus_tw:
        print (k, v)

th 218
sweden 217
owner 217
channel 217
govt 217
burn 217
websit 217
marri 217
bar 216
cancer 216
sun 216
prove 216
korea 216
press 216
steve 215
israel 215
swear 215
tip 215
insult 215
border 215
democrat 214
produc 214
main 214
hop 214
soft 214
talent 214
daughter 213
click 213
camera 213
po 213
store 213
jail 212
rich 210
a.m. 210
roll 209
mine 209
rude 209
modi 209
note 209
confus 208
becaus 208
green 208
direct 208
villanova 208
congrat 208
bitcoin 208
struggl 207
whoever 207
roseann 206
town 206
innoc 206
marvin 206
york 205
re 205
guest 205
rain 205
basic 204
hero 204
center 204
door 204
kick 203
destroy 203
happyeast 203
area 203
htt 203
boo 203
drunk 203
fund 202
yodel 202
opinion 202
cost 202
qualiti 202
crack 202
ok 202
advic 202
hide 201
vacat 201
hug 201
terrorist 201
token 200
hunt 200
yall 200
remov 199
scari 199
provid 199
forward 199
imag 199
career 199
form 198
mental 198
band 198
fat 198
honor 198
download 197
popular 197
repeat 197
headshot 196
him 196
donat 196
med

In [205]:
set(corpus0) - set(corpus_tw)

{'warcraft',
 'roulett',
 'contin',
 'camper',
 'mcmahon',
 'europa',
 'wrench',
 'hanzo',
 'kernel',
 'peripher',
 'centiped',
 'tx',
 'infest',
 'annihil',
 'muh',
 'heresi',
 'ottoman',
 'courteou',
 'chassi',
 'metaphor',
 'okc',
 'midrang',
 'incivil',
 'recharg',
 'calib',
 'rotat',
 'accordingli',
 'gunshot',
 'watcher',
 'dani',
 'hindsight',
 'uncivil',
 'quo',
 'antenna',
 'plural',
 'npc',
 'm1',
 'hefti',
 'arduino',
 'wiggl',
 'idl',
 'rw',
 'psych',
 'playmak',
 'acl',
 'gill',
 'inclin',
 'hk',
 'folder',
 'regener',
 'cach',
 'forerunn',
 'grandchild',
 'fianc',
 'debri',
 'excit',
 'hassl',
 'nip',
 'cohes',
 'unnecessarili',
 'itt',
 'cp',
 'psychot',
 'globaloffens',
 'capita',
 'tippr',
 'atk',
 'colin',
 'neurosci',
 'comet',
 'wal',
 'gloss',
 '4k',
 'spous',
 'inquisit',
 'decapit',
 'apathi',
 'expect',
 'renegad',
 'wolv',
 'ptr',
 'msi',
 'sol',
 'rooster',
 'concess',
 'baffl',
 'weaken',
 'sleeper',
 'reload',
 'fallaci',
 'bpd',
 'zap',
 'spamm',
 'geograph

# Use TF-IDF to train classifier

# Construct word space

In [216]:
def get_dic(corpus):
    word_space=defaultdict(int)
    for word in corpus:
        if word not in word_space:
            word_space[word]=len(word_space) 
    print ("The number of corpus is " + str(len(word_space)))
    return word_space

In [217]:
corpus = corpus0 + corpus1
word_space = get_dic(corpus)

The number of corpus is 16657


In [ ]:
# save the word_space for later use
outfile = json.dumps(word_space)
f = open("D:/Depression_project/word_space.json","w")
f.write(outfile)
f.close()

In [ ]:
def get_sparse_vec(docs, word_space):
    sparse_vecs = []
    for doc in docs:
        # create empty vector
        sparse_vec = np.zeros(len(word_space))
        for word in doc.split():
            if word in word_space:
                sparse_vec[word_space[word]] = 1
        sparse_vecs.append(sparse_vec) 
    print ("The number of samples is " +str(len(sparse_vecs)))
    if len(sparse_vecs[0]) != len(word_space):
        print ("error")
    return sparse_vecs

In [218]:
def get_freq_vec(docs, word_space):
    freq_vecs = []
    for doc in docs:
        # create empty vector
        freq_vec = np.zeros(len(word_space))
        for word in doc.split():
            if word in word_space:
                freq_vec[word_space[word]] += 1
        freq_vecs.append(freq_vec) 
    print ("The number of samples is " +str(len(freq_vecs)))
    if len(freq_vecs[0]) != len(word_space):
        print ("error")
    return freq_vecs   

In [220]:
freq_vecs0 = get_freq_vec(text_control[:5], word_space)

The number of samples is 5


In [230]:
tfidf_transformer = TfidfTransformer(use_idf=False).fit(freq_vecs0)
X0_tfidf = tfidf_transformer.transform(freq_vecs0)
print (X0_tfidf.shape)

# LabMT

LabMT is a word list score for sentiment analysis. It is available as supplementary material from an article:

http://www.plosone.org/article/fetchSingleRepresentation.action?uri=info:doi/10.1371/journal.pone.0026752.s001

In [1]:
import pandas as pd
 
url = 'http://www.plosone.org/article/fetchSingleRepresentation.action?uri=info:doi/10.1371/journal.pone.0026752.s001'
labmt = pd.read_csv(url, skiprows=2, sep='\t', index_col=0)

In [2]:
average = labmt.happiness_average.mean()
happiness = (labmt.happiness_average - average).to_dict()
 
def score(text):
    words = text.split()
    return sum([happiness.get(word.lower(), 0.0) for word in words]) / len(words)

print (score('Incredibly and excellent'))
print (score('Oh so bad should be avoided at all costs'))

1.2647603208765839
-0.6663507902345269


In [10]:
for t in text_control[:10]:
    print ("Sentiment score: "+str(score(t))+' '+t)

Sentiment score: 0.20512493238347118 want see how would address what predat eat food none charact eat meat entir movi be slightli bother bojack horseman do fantast dark episod show differ chicken eat chicken that be function member societi
Sentiment score: 0.024760320876583936 how tall be
Sentiment score: 0.08122313217992934 live about mile capit be weird man
Sentiment score: -0.4538542537485599 be not pretend dude have other fuck altern call work take dick ass avoid then be gay noth wrong be what be
Sentiment score: 0.1680936542099174 do not make use be sure
Sentiment score: 0.07799964199039532 comrad friendli remind subreddit be space socialist have question want debat consid sub creat specif /r socialism_101 /r /r debatecommunism/ be also encourag use search function search topic may not be well vers may have be cover extens befor acquaint rule sidebar comment post rule be strictli enforc non subscrib person attack harass will not be toler bigotri ableism hate speech will be meet im

In [11]:
for t in text_depr[:10]:
    print ("Sentiment score: "+str(score(t))+' '+t)

Sentiment score: 0.19703580512623212 anybodi els look find reason live can not find
Sentiment score: -0.1258347192329889 bad part be have live crazi fuck world
Sentiment score: 0.3113002673971535 what do everyon do their free time summer break colleg blue seemingli my friend can not stand long summer break colleg bit month leav lot free time even work long shift week work anybodi els find case be say anybodi have idea how can fill my free time there be onli so much exercis netflix cook person can do haha
Sentiment score: 0.05721392677499744 hate peopl reaction when tell have depress have alreadi make huge leap admit peopl care have depress then go all need do solv your problem do not ask your advic just want let know have take lot courag tell be alreadi see doctor do not need your workout socialis more yoghurt advic cure my major depress want let know care just want someon understand what be go be sorri be just so angri want kill just make feel bad
Sentiment score: 0.08095107531718422 

In [ ]:
labmt